In [1]:
from reddit_keys import reddit_user, reddit_pw, reddit_secret_token, reddit_clientID
import praw
import pandas as pd
import numpy as np
import pickle
import json
from datetime import datetime
import os.path
from threading import Timer
from time import sleep

In [2]:
#Function

class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False

In [3]:
og_files = ['data_hot.json', 'data_hot.pkl']
for file in og_files:
    if os.path.exists(file):
        print("File Exists. Bypassing First_Run")
    else:
        print("No Exist") 

File Exists. Bypassing First_Run
File Exists. Bypassing First_Run


In [4]:
def pull_posts(sub_reddit, reddit_user, reddit_pw, reddit_secret_token, reddit_clientID):
    # Submit credentials and access api
    reddit = praw.Reddit(client_id = reddit_clientID,
                     client_secret = reddit_secret_token,
                     user_agent = 'WSB_search',
                     username = reddit_user,
                     password = reddit_pw)
    # Create pickle file of sub_id's for serialization to avoid duplicate post pulls
    # Hey Morty, I turned myself into a pickle! im_pkl_dict!!    
    with open('data_hot.pkl','rb') as pickle_file:
        im_pkl_dict=pickle.load(pickle_file)

    sub_wsb = reddit.subreddit(sub_reddit)
    top_sub = sub_wsb.hot(limit=60)
    sub_dict = {}
    sub_id = []
    sub_words = []
    sub_time = []
    sub_upv = []

    #Access list of words from top titles in the sub 
    for sub in top_sub:
        s_id = sub.id
        title_1 = sub.title
        time = sub.created_utc
        s_id =sub.id
        up_votes = sub.score 
        if s_id not in im_pkl_dict:
            sub_id.append(s_id)
            sub_words.append(title_1)
            sub_time.append(time)
            sub_upv.append(up_votes)
            im_pkl_dict.append(s_id)
        else:
            continue


    print('errythang pulled son')
    #Check to make sure files are different
    print(len(sub_id))
    print(len(im_pkl_dict))
    print(len(sub_id), len(sub_words), len(sub_time))

    ## Append pkl file
    with open('data_hot.pkl','wb') as pickle_file:
        pickle.dump(im_pkl_dict, pickle_file)


    ## Reload JSON file for conversion
    with open('data_hot.json','r') as json_file:
        data = json.load(json_file)
    print("Last Key Number of data : " + list(data['id'].keys())[-1])
    print("Number of entries: " + str(int(list(data['id'].keys())[-1])+1))
    
    #Keys and Values Lists for dict comprehensions
    keys = ['id','title','time','up_votes']
    values = [sub_id, sub_words, sub_time,sub_upv]
    
    #Dict Comprehensions
    id_dict = {str(num + int(list(data['id'].keys())[-1])+1): sub_id[num] for num in range(len(sub_id))}
    word_dict = {str(num + int(list(data['title'].keys())[-1])+1): sub_words[num].split() for num in range(len(sub_words))}
    time_dict = {str(num + int(list(data['time'].keys())[-1])+1): sub_time[num] for num in range(len(sub_time))}
    upv_dict = {str(num + int(list(data['up_votes'].keys())[-1])+1): sub_upv[num] for num in range(len(sub_upv))}
    #Combine dictionaries using dict comprehension to create JSON compatible file
    list_o_dicts = [id_dict, word_dict, time_dict,upv_dict]
    sub_dict['posts'] = {keys[num]: list_o_dicts[num] for num in range(len(list_o_dicts))}
    dict_entries = sub_dict['posts']
    
    #Append pulled dictionaries to JSON data
    for key in dict_entries.keys():
        data[key].update(dict_entries[key]) 

    ## Append JSON file
    with open('data_hot.json','w') as json_file:
        json.dump(data, json_file)


In [ ]:
#### We'll start off with 60 pulls every 2 minutes per day
# 86400 seconds per day
# so we'll run this pull every 864 seconds
rt = RepeatedTimer(300, pull_posts, "wallstreetbets",reddit_user, reddit_pw, reddit_secret_token, reddit_clientID) # it auto-starts, no need of rt.start()
try:
    sleep(86400) # your long-running job goes here... Sleep after 3.5 Hours
finally:
    rt.stop() # better in a try/finally block to make sure the program ends!

errythang pulled son
37
1973
37 37 37
Last Key Number of data : 1935
Number of entries: 1936
errythang pulled son
2
1975
2 2 2
Last Key Number of data : 1972
Number of entries: 1973
errythang pulled son
0
1975
0 0 0
Last Key Number of data : 1974
Number of entries: 1975
errythang pulled son
0
1975
0 0 0
Last Key Number of data : 1974
Number of entries: 1975
errythang pulled son
2
1977
2 2 2
Last Key Number of data : 1974
Number of entries: 1975
errythang pulled son
1
1978
1 1 1
Last Key Number of data : 1976
Number of entries: 1977
errythang pulled son
0
1978
0 0 0
Last Key Number of data : 1977
Number of entries: 1978
errythang pulled son
2
1980
2 2 2
Last Key Number of data : 1977
Number of entries: 1978
